In [1]:
import tkinter as tk
import random
import numpy as np

class Game2048:
    def __init__(self):
        self.grid = np.zeros((4, 4), dtype=int)
        self.score = 0
        self.add_new_tile()
        self.add_new_tile()
        
        # Create the game window
        self.window = tk.Tk()
        self.window.title("2048 Game")
        self.window.bind("<Key>", self.key_press)

        self.frame = tk.Frame(self.window)
        self.frame.grid()
        
        # Create tiles for the game board
        self.cells = [[tk.Label(self.frame, text="", width=6, height=3, font=("Helvetica", 24), borderwidth=2, relief="groove") for _ in range(4)] for _ in range(4)]
        for i in range(4):
            for j in range(4):
                self.cells[i][j].grid(row=i, column=j, padx=5, pady=5)
        
        # Create score label
        self.score_label = tk.Label(self.window, text=f"Score: {self.score}", font=("Helvetica", 18))
        self.score_label.grid()

        self.update_grid()
    
    def add_new_tile(self):
        x, y = random.choice(list(zip(*np.where(self.grid == 0))))
        self.grid[x][y] = 2 if random.random() < 0.9 else 4

    def move(self, direction):
        if direction == 'up':
            self.grid = self._move_up()
        elif direction == 'down':
            self.grid = self._move_down()
        elif direction == 'left':
            self.grid = self._move_left()
        elif direction == 'right':
            self.grid = self._move_right()

        if np.any(self.grid == 0):
            self.add_new_tile()

        self.update_grid()

        if self.is_game_over():
            self.game_over()

    def _move_left(self):
        new_grid = np.zeros((4, 4), dtype=int)
        for i in range(4):
            merged_line, score_add = self._merge_line(self.grid[i, :])
            new_grid[i, :len(merged_line)] = merged_line
            self.score += score_add
        return new_grid

    def _move_right(self):
        new_grid = np.zeros((4, 4), dtype=int)
        for i in range(4):
            merged_line, score_add = self._merge_line(self.grid[i, ::-1])
            new_grid[i, -len(merged_line):] = merged_line[::-1]
            self.score += score_add
        return new_grid

    def _move_up(self):
        self.grid = self.grid.T
        new_grid = self._move_left()
        self.grid = new_grid.T
        return self.grid

    def _move_down(self):
        self.grid = self.grid.T
        new_grid = self._move_right()
        self.grid = new_grid.T
        return self.grid

    def _merge_line(self, line):
        non_zero_line = line[line != 0]
        new_line = []
        score_add = 0
        skip = False
        for i in range(len(non_zero_line)):
            if skip:
                skip = False
                continue
            if i != len(non_zero_line) - 1 and non_zero_line[i] == non_zero_line[i + 1]:
                new_line.append(2 * non_zero_line[i])
                score_add += 2 * non_zero_line[i]
                skip = True
            else:
                new_line.append(non_zero_line[i])
        return np.array(new_line), score_add

    def is_game_over(self):
        if not 0 in self.grid:
            for i in range(4):
                for j in range(3):
                    if self.grid[i][j] == self.grid[i][j + 1] or self.grid[j][i] == self.grid[j + 1][i]:
                        return False
            return True
        return False

    def update_grid(self):
        for i in range(4):
            for j in range(4):
                value = self.grid[i][j]
                self.cells[i][j].config(text="" if value == 0 else str(value), bg=self.get_tile_color(value))

        self.score_label.config(text=f"Score: {self.score}")
        self.window.update_idletasks()

    def get_tile_color(self, value):
        tile_colors = {
            0: "#CDC1B4", 2: "#EEE4DA", 4: "#EDE0C8", 8: "#F2B179", 16: "#F59563",
            32: "#F67C5F", 64: "#F65E3B", 128: "#EDCF72", 256: "#EDCC61", 512: "#EDC850",
            1024: "#EDC53F", 2048: "#EDC22E"
        }
        return tile_colors.get(value, "#CDC1B4")

    def key_press(self, event):
        key = event.keysym.lower()
        if key in ['up', 'down', 'left', 'right']:
            self.move(key)

    def game_over(self):
        game_over_window = tk.Toplevel(self.window)
        game_over_window.title("Game Over")
        tk.Label(game_over_window, text="Game Over", font=("Helvetica", 18)).pack()
        tk.Label(game_over_window, text=f"Final Score: {self.score}", font=("Helvetica", 14)).pack()

        def close_game():
            game_over_window.destroy()
            self.window.destroy()

        tk.Button(game_over_window, text="Exit", command=close_game).pack()

    def run(self):
        self.window.mainloop()

# Run the game
game = Game2048()
game.run()
